# Running a QiskitPattern as a function

In this tutorial, we will write a basic QiskitPattern using Qiskit Serverless. We will show how to run the pattern remotely and retrieve the results from the serverless client.

### Writing the QiskitPattern

First, we need to write the pattern code and save it to a file called [pattern.py](./source_files/pattern.py). This pattern creates a two-qubit quantum circuit that prepares a Bell state, measures the result, and saves the measured probability distribution.

The code for the pattern is shown below:

```python
# source_files/pattern.py

from qiskit import QuantumCircuit
from qiskit.primitives import Sampler

from qiskit_serverless import save_result

# all print statement will be available in job logs
print("Running pattern...")

# creating circuit
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()

# running Sampler primitive
sampler = Sampler()
quasi_dists = sampler.run(circuit).result().quasi_dists

# save results of pattern execution, 
# which will be accessible by calling `.result()`
save_result(quasi_dists)
print("Completed running pattern.")
```

### Deploying the function

To run the pattern, we need to import the necessary classes and configure them. One of these classes is `ServerlessClient`, which is a client class for interacting with compute resources.

 The client stores configuration information about our compute resources, such as where they are located and how to connect to them. In this example, we will use a provider that is connected to a local Docker Compose setup. In this case, it allows us to run the pattern locally on our machine. If you want to run the pattern elsewhere, you will need to provide the corresponding host and authentication details.

In [1]:
from qiskit_serverless import ServerlessClient
import os

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit.github.io/qiskit-serverless/deployment/local.html).

In [2]:
client = ServerlessClient(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

client

<gateway-client>



`QiskitFunction` accepts couple of required parameters:

- title - name of the program
- entrypoint - name of python file you want to execute
- working_dir - directory where  your script is located (directory size must be less than 50MB). This is optional parameter and will be current folder by default.

> Warning! All content of `working_dir` will be shipped to cluster for execution

> Warning! Execution of `upload` function ships All content of `working_dir`.  When the contents of `working_dir` is changed, the `upload` function must be called again to update the shipped directory contents.

In [3]:
from qiskit_serverless import QiskitFunction

function = QiskitFunction(
    title="my-first-pattern", entrypoint="pattern.py", working_dir="./source_files/"
)

client.upload(function)

'my-first-pattern'

### Running the QiskitFunction

After deploying the QiskitFunction, we can see our pattern in a `list` of availiable functions.

In [4]:
my_pattern_function = client.get("my-first-pattern")
my_pattern_function

QiskitFunction(my-first-pattern)

We can run any function by calling `fun` method on function object.

In [5]:
job = my_pattern_function.run()
job

<Job | 50aaf775-7b14-40c9-bf35-51cb1f88fbe2>

[Job](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.core.Job.html#qiskit_serverless.core.Job) instances have a `status()` method to check status of pattern execution.

In [7]:
job.status()

'QUEUED'

`Job` instances also have a `result()` method for retrieving results. The `result()` method will not return until the job is done running the pattern.

In [ ]:
job.result()

To inspect the logs from a pattern, access them from the ``Job`` instance.

In [8]:
print(job.logs())

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
Running pattern...
Completed running pattern.



`ServerlessClient` object has method `.widget` which renders Jupyter widget to see list of executed programs.

In [8]:
client.widget()